In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# COMPLETE HOUSE PRICE PREDICTION - ONE DAY SOLUTION
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 1. LOAD DATA
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

# 2. SELECT FEATURES (Most important numerical only)
features = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'GarageArea', 
           'YearBuilt', 'OverallQual', 'FullBath', 'BedroomAbvGr']

# 3. PREPROCESS
def preprocess(df):
    df_clean = df[features].copy()
    # Fill missing values with median
    for col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    # Add engineered features
    df_clean['House_Age'] = 2023 - df_clean['YearBuilt']
    df_clean['Total_SF'] = df_clean['GrLivArea'] + df_clean['TotalBsmtSF']
    return df_clean

X_train = preprocess(train)
X_test = preprocess(test)
y_train = np.log(train['SalePrice'])  # Log transform target

# 4. TRAIN MODEL
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Ridge(alpha=10)
model.fit(X_train_scaled, y_train)

# 5. PREDICT AND SUBMIT
predictions = np.exp(model.predict(X_test_scaled))  # Transform back from log

submission = pd.DataFrame({
    'Id': test['Id'], 
    'SalePrice': predictions
})
submission.to_csv('submission.csv', index=False)

print("✅ DONE! Submission file created!")
print(f"Predicted prices: ${predictions.min():,.0f} - ${predictions.max():,.0f}")

✅ DONE! Submission file created!
Predicted prices: $63,929 - $1,285,331


In [3]:
predictions = np.maximum(predictions, 0)
train['SalePrice'].describe()
# Output:
# min    34900.000000
# max   755000.000000
# mean  180921.195890

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64